# Sarcasm Detection
 **Acknowledgement**

Misra, Rishabh, and Prahal Arora. "Sarcasm Detection using Hybrid Neural Network." arXiv preprint arXiv:1908.07414 (2019).

**Required Files given in below link.**

https://drive.google.com/drive/folders/1xUnF35naPGU63xwRDVGc-DkZ3M8V5mMk

## Install `Tensorflow2.0` 

In [1]:
#!!pip uninstall tensorflow
#!pip install tensorflow==2.0.0

## Get Required Files from Drive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, drive, files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
project_path = '/content/drive/My Drive/Bidi_LSTM_Project/'

In [4]:
#!cd project_path

#**## Reading and Exploring Data**

## Read Data "Sarcasm_Headlines_Dataset.json". Explore the data and get  some insights about the data. ( 4 marks)
Hint - As its in json format you need to use pandas.read_json function. Give paraemeter lines = True.

In [5]:
import pandas as pd
import json
import numpy as np
downloaded = drive.CreateFile({'id':"17VjjLXQFE26E4OOCiJgUlRLTv_0x1Ga0"})
downloaded.GetContentFile('Sarcasm_Headlines_Dataset.json')
Data_df=pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)
Data_df.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


## Drop `article_link` from dataset. ( 2 marks)
As we only need headline text data and is_sarcastic column for this project. We can drop artical link column here.

In [6]:
sarcasm_df=Data_df.drop(['article_link'],axis=1) 
sarcasm_df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


## Get the Length of each line and find the maximum length. ( 4 marks)
As different lines are of different length. We need to pad the our sequences using the max length.

In [7]:
for c in sarcasm_df:
    if sarcasm_df[c].dtype == 'object':
        print('Max length of column %s: %s\n' %  (c, sarcasm_df[c].map(len).max()))

Max length of column headline: 254



#**## Modelling**

## Import required modules required for modelling.

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

# Set Different Parameters for the model. ( 2 marks)

In [9]:
max_features = 10000
maxlen = 254
embedding_size = 200

## Apply Keras Tokenizer of headline column of your data.  ( 4 marks)
Hint - First create a tokenizer instance using Tokenizer(num_words=max_features) 
And then fit this tokenizer instance on your data column df['headline'] using .fit_on_texts()

In [10]:
token=Tokenizer(num_words=max_features)

# Define X and y for your model.

In [11]:
X = token.texts_to_sequences(sarcasm_df['headline'])
X = pad_sequences(X, maxlen = maxlen)
y = np.asarray(sarcasm_df['is_sarcastic'])

print("Number of Samples:", len(X))
print(X[0])
print("Number of Labels: ", len(y))
print(y[0])

Number of Samples: 26709
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Number of Labels:  26709
0


In [12]:
sequences = token.texts_to_sequences(sarcasm_df['headline'])
data = pad_sequences(sequences, maxlen= maxlen) 
print(data) 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Get the Vocabulary size ( 2 marks)
Hint : You can use tokenizer.word_index.

In [13]:
vocab_size=token.word_index
print(vocab_size)

{}


#**## Word Embedding**

## Get Glove Word Embeddings

In [14]:
glove_file = 'glove.6B.zip'

In [15]:
downloaded = drive.CreateFile({'id':"1X65WCZ-rilw4eMP-b_DUltoeyRlWL2mI"})
downloaded.GetContentFile(glove_file)

#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

# Get the Word Embeddings using Embedding file as given below.

In [16]:
EMBEDDING_FILE = 'glove.6B.200d.txt'
downloaded = drive.CreateFile({'id':"1QOfuJdgsOSZtY1ZuwPoJ3nKwzWCtrd17"})
downloaded.GetContentFile(EMBEDDING_FILE)

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

# Create a weight matrix for words in training docs

In [17]:
embedding_matrix = np.zeros((maxlen, 200))

for word, i in token.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

## Create and Compile your Model  ( 7 marks)
Hint - Use Sequential model instance and then add Embedding layer, Bidirectional(LSTM) layer, then dense and dropout layers as required. 
In the end add a final dense layer with sigmoid activation for binary classification.


In [18]:
model = Sequential()

In [19]:
### Embedding layer for hint 
model.add(Embedding(maxlen, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         50800     
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         336896    
_________________________________________________________________
dense (Dense)                (None, None, 1)           257       
Total params: 387,953
Trainable params: 387,953
Non-trainable params: 0
_________________________________________________________________


# Fit your model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy ( 5 marks)


In [22]:
batch_size = 100
epochs = 5

## Add your code here ##
history = model.fit(X, y, epochs=epochs, batch_size=batch_size,validation_split=0.2, verbose=1)

Epoch 1/5
214/214 [==============================] - 395s 2s/step - loss: 0.6867 - accuracy: 0.5593 - val_loss: 0.6852 - val_accuracy: 0.5680
Epoch 2/5
214/214 [==============================] - 393s 2s/step - loss: 0.6865 - accuracy: 0.5593 - val_loss: 0.6850 - val_accuracy: 0.5680
Epoch 3/5
214/214 [==============================] - 393s 2s/step - loss: 0.6866 - accuracy: 0.5593 - val_loss: 0.6840 - val_accuracy: 0.5680
Epoch 4/5
214/214 [==============================] - 393s 2s/step - loss: 0.6864 - accuracy: 0.5593 - val_loss: 0.6842 - val_accuracy: 0.5680
Epoch 5/5
214/214 [==============================] - 393s 2s/step - loss: 0.6862 - accuracy: 0.5593 - val_loss: 0.6841 - val_accuracy: 0.5680
